In [1]:
%matplotlib inline
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from matplotlib.pyplot import figure
import plotly.graph_objects as go

In [2]:
def getMerics(func,size:int,mu=0):
    if(func == st.poisson):
        array = func.rvs(size=size,mu=mu)
    else:
        array = func.rvs(size = size)
    array.sort()
    # выборочное среднее
    x_av = sum(array)/len(array)

    # выборочная медиана
    if(len(array) % 2 == 0):
        x_med = (array[len(array)//2] + array[len(array)//2 - 1])/2
    else:
        x_med = array[len(array)//2]

    # Полусумма экстремальных выбросов
    z_r = (array[0] + array[len(array) - 1])/2


    def cvart(p,array):
        # на вход sort list
        l = len(array)*p
        if(int(l) != l):
            return array[int(l) + 1]
        else:
            return array[int(l)]

    # Полусумма квартилей
    zq = (cvart(1/4,array) + cvart(3/4,array))/2

    def sum_(array,start,end):
        sm = 0
        for i in range(start, end):
            sm += array[i]
        return sm

    # усеченное среднее
    r = len(array)//4
    n = len(array)
    z_tr = 1/(n - 2*r) * (sum_(array,r,n - r - 1))


    return np.array([x_av,x_med,z_r,zq,z_tr])

In [3]:
def getMean(array):
    return array.mean()

def getDis(array):
    sm = 0
    mn = getMean(array)
    for i in array:
        sm += (i - mn) ** 2
    return sm/ len(array)

In [8]:
def getMetricsRepeat(func,size,mu=0):
    ar = [getMerics(func,size,mu = mu)]
    for i in range(999):
        ar = np.concatenate(([getMerics(func,size,mu=mu)],ar),axis = 0)
    disp = []
    mean = []
    mean_m_disp = []
    mean_p_disp = []
    for i in range(len(ar[0])):
        mean.append(getMean(ar[:,i]))
        disp.append(round(getDis(ar[:,i]),2))
        mean_m_disp.append(mean[i] - disp[i] ** 0.5)
        mean_p_disp.append(mean[i] + disp[i] ** 0.5)
    return {'mean':mean,
            'disp' : disp,
           'mean + sqrt(d)' : mean_m_disp,
           'mean - sqrt(d)' : mean_p_disp
           }

In [14]:
def Draw(func,size, mu = 0):
    d = getMetricsRepeat(func,size,mu=mu)
    vals = ['x_av','x_med','z_r','zq','z_tr']
    col = [['mean'] + ['disp'] + ['mean - sqrt(d)'] + ['mean + sqrt(d)']]
    for i in range(0,len(d['mean'])):
        col.append([d['mean'][i]] + [d['disp'][i]] + [d['mean + sqrt(d)'][i]] + 
                  [d['mean - sqrt(d)'][i]])
    fig = go.Figure(data=[go.Table(
        header=dict(values=['size = ' +str(size)] + vals,
                    line_color='darkslategray',
                    fill_color='lightskyblue',
                    align='left',
         ),
        cells=dict(values=col, # 2nd column
                   line_color='darkslategray',
                   fill_color='lightcyan',
                   align='left',
                  format = ["",".3f"]))
    ])

    fig.update_layout(width=1000, height=300)
    fig.show()

In [15]:
# нормальное распределение 
Draw(st.norm,10)
Draw(st.norm,100)
Draw(st.norm,1000)

In [20]:
# Распределение Коши
Draw(st.cauchy,10)
Draw(st.cauchy,100)
Draw(st.cauchy,1000)

In [17]:
# распределение Лапласа
Draw(st.laplace,10)
Draw(st.laplace,100)
Draw(st.laplace,1000)

In [21]:
# Распределение Пуассона
Draw(st.poisson,10,mu=4)
Draw(st.poisson,100,mu=4)
Draw(st.poisson,1000,mu=4)

In [19]:
# равномерное Распределение
Draw(st.uniform,10)
Draw(st.uniform,100)
Draw(st.uniform,1000)